## Ejemplo clasificatorio sobre enfermos de cancer

Importo los datos de kaggle "Breast Cancer Wisconsin";<p> https://www.kaggle.com/uciml/breast-cancer-wisconsin-data/data <p>
Información sobre los datos en : <p>
https://archive.ics.uci.edu/ml/machine-learning-databases/breast-cancer-wisconsin/wdbc.names<p> <p>
Tomo como $y=1$ la diagnosis de un tumor maligno. 
Observo las variables de los 10 primeros individuos.

In [1]:
import pandas as pd
import numpy as np

data = pd.read_csv('data.csv').ix[:, 1:32]
data['diagnosis']=data['diagnosis'].map({'M':1,'B':0})

Primero divido la muestra en entrenamiento, validación y test. (60/20/20) <p>
Calculo distintos modelos variando C. <p>
Calculo para la Accuracy para mis tres data sets.<p>
Calculo el recall y precisión de cada modelo para el test set.

In [2]:
from sklearn import linear_model, metrics, cross_validation

train, validate, test = np.split(data.sample(frac=1, random_state=1),
                                 [int(.6*len(data)), int(.8*len(data))])
list_C = np.arange(1/10000000 , 1000, 0.1)
score_train = np.zeros(len(list_C)); score_val = np.zeros(len(list_C))
score_test = np.zeros(len(list_C)) ; recall_test = np.zeros(len(list_C))
precision_test= np.zeros(len(list_C)); count = 0
for C in list_C:
    reg = linear_model.LogisticRegression(C=C)
    reg.fit(train.ix[:,2:32], train['diagnosis'])
    score_train[count]= metrics.accuracy_score(
        train['diagnosis'], reg.predict(train.ix[:, 2:32]))
    score_val[count] = metrics.accuracy_score(
        validate['diagnosis'], reg.predict(validate.ix[:, 2:32]))
    score_test[count] = metrics.accuracy_score(
        test['diagnosis'], reg.predict(test.ix[:, 2:32]))
    recall_test[count] = metrics.recall_score(
        test['diagnosis'], reg.predict(test.ix[:, 2:32]))
    precision_test[count] = metrics.precision_score(
        test['diagnosis'], reg.predict(test.ix[:, 2:32]))
    count = count + 1 

Creo un data frame para cada modelo (distinto valor de C) con todas las métricas calculadas.

In [3]:
matrix = np.matrix(np.c_[list_C, score_train, score_val, 
                         score_test, recall_test, precision_test])
models = pd.DataFrame(data = matrix, columns = 
             ['C', 'Train Accuracy', 'Validation Accuracy', 
              'Test Accuracy', 'Test Recall', 'Test Precision'])
models.ix[:, :4].head(n=5)

,C,Train Accuracy,Validation Accuracy,Test Accuracy
0,1.000000e-07,0.369501,0.359649,0.394737
1,1.000001e-01,0.926686,0.973684,0.973684
2,2.000001e-01,0.932551,0.964912,0.964912
3,3.000001e-01,0.935484,0.964912,0.964912
4,4.000001e-01,0.938416,0.964912,0.964912


Elijo el mejor modelo en función de la Accuracy en el validate test. Observo el resto de métricas.

In [4]:
best_index = models['Validation Accuracy'].idxmax()
models.ix[best_index, :]

C                      399.400000
Train Accuracy           0.982405
Validation Accuracy      0.991228
Test Accuracy            0.964912
Test Recall              0.933333
Test Precision           0.976744
Name: 3994, dtype: float64

El modelo lo hemos elegido tomando como parámetro el validate set. <p>
La manera correcta de estimar la capacidad de generalización es evaluarla sobre el test set. <p>
Si lo hicieramos sobre el validate test, sobreestimariamos su capacidad al haberlo tomado como parámetro al elegir el modelo.

In [5]:
reg = linear_model.LogisticRegression(C=list_C[best_index])
reg.fit(train.ix[:,2:32], train['diagnosis'])

LogisticRegression(C=399.40000010000006, class_weight=None, dual=False,
          fit_intercept=True, intercept_scaling=1, max_iter=100,
          multi_class='ovr', n_jobs=1, penalty='l2', random_state=None,
          solver='liblinear', tol=0.0001, verbose=0, warm_start=False)

Calculo las matrices de confusión.

In [6]:
print('Train Set')
m_confusion_train = metrics.confusion_matrix(train['diagnosis'],
                         reg.predict(train.ix[:, 2:32]))
pd.DataFrame(data = m_confusion_train, 
            columns = ['Predicted 1', 'Predicted 0'],
            index = ['Actual 1', 'Actual 0'])

Train Set


,Predicted 1,Predicted 0
Actual 1,213,2
Actual 0,4,122


In [7]:
print('Validate Set')
m_confusion_validate = metrics.confusion_matrix(validate['diagnosis'],
                         reg.predict(validate.ix[:, 2:32]))
pd.DataFrame(data = m_confusion_validate, 
            columns = ['Predicted 1', 'Predicted 0'],
            index = ['Actual 1', 'Actual 0'])

Validate Set


,Predicted 1,Predicted 0
Actual 1,73,0
Actual 0,1,40


In [8]:
print('Test Set')
m_confusion_test = metrics.confusion_matrix(test['diagnosis'],
                         reg.predict(test.ix[:, 2:32]))
pd.DataFrame(data = m_confusion_test, 
             columns = ['Predicted 1', 'Predicted 0'],
             index = ['Actual 1', 'Actual 0'])

Test Set


,Predicted 1,Predicted 0
Actual 1,68,1
Actual 0,3,42


#### Mi modelo comete más error de tipo 1 y menos de tipo 2:

#### Error tipo 1 : Clasificar pacientes que tiene un tumor maligno cuando es benigno. (Un gran shock inicial)

#### Error tipo 2 : Clasificar pacientes con tumor benigno cuando es maligno. (Mayor probabilidad de muerte)

#### Parece razonable.